In [1]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc
from datetime import datetime, timedelta

In [2]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password

Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [3]:
Defaul_Column_Pedidos_dia=['Documento', 'GTr', 'Denomin.', 'ClVt', 'Denominación', 'Solic.',
       'Creado el', 'Fecha doc.', 'Pedido', 'Func.', 'Responsab', 'Creado',
       'OrgVt', 'CDis', 'Se', 'OfVta', 'GVen', 'Mon.', 'Valor neto']

In [4]:
# PEDIDOSEXITO


def Search_VA05N(Transsaccion,session,provision,variant):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    return(session)

In [5]:
Transsaccion='va05n'
provision='PEDIDOSEXITO'
variant='JESPINOSAP'

Search_VA05N(Transsaccion,session,provision,variant)

<COMObject <unknown>>

In [6]:
Name_VA05N="Pedidos_diarios"
Ruta_Exito_ZSD010_ZVMI="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_VA05N,session,Ruta_Exito_ZSD010_ZVMI)

In [7]:
Pedidos_VN05N=pd.read_csv(Ruta_Exito_ZSD010_ZVMI+Name_VA05N+".txt",delimiter="\t",skiprows=1)
Pedidos_VN05N=Funtions.Clean_Columns(Pedidos_VN05N)
Pedidos_VN05N=Funtions.default_column(Defaul_Column_Pedidos_dia,Pedidos_VN05N)
Pedidos_VN05N=Pedidos_VN05N[Pedidos_VN05N['Pedido'].notnull()]

In [8]:
Agenda=["85","20","146","149","50","138","45"]

Pedidos_VN05N['Pedido']=Funtions.Estandarizo_Pedidos(Pedidos_VN05N['Pedido'])
Pedidos_VN05N['Pedido']=Funtions.complete_pedidos(Pedidos_VN05N['Pedido'],Agenda)


In [9]:
Lunes=["0085"]


buscar=Funtions.Search_Agenda_Exito(Pedidos_VN05N['Pedido'],Lunes)
filtro_cencosub=Pedidos_VN05N[Pedidos_VN05N['Pedido'].isin(buscar)]

In [12]:
filtro_cencosub['Pedido'].drop_duplicates().value_counts()

0085759670    1
0085768083    1
0085768217    1
0085768085    1
0085768086    1
             ..
0085761893    1
0085761913    1
0085762016    1
0085762132    1
0085769571    1
Name: Pedido, Length: 159, dtype: int64

In [65]:
session.findById("wnd[0]/usr/btn%_SO_DOC_E_%_APP_%-VALU_PUSH").press()

In [13]:
def Search_VN05N(Transsaccion,Series,session):      #(column Dataframe)
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    provision: disposicion de interes
    """
    session.StartTransaction(Transsaccion)
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SO_DOC_E_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()


In [79]:
Search_VN05N('zsd79',filtro_cencosub['Pedido'].drop_duplicates(),session)

In [94]:
Series=filtro_cencosub['Pedido'].drop_duplicates().reset_index(drop=True)
session.StartTransaction('zsd79')
Series=Series.to_clipboard(index=False, header=False)
session.findById("wnd[0]/usr/btn%_SO_DOC_E_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
